In [ ]:
import pandas as pd
from google.colab import files

# Upload the Excel file
uploaded = files.upload()

# Load the uploaded file into a pandas DataFrame
file_name = next(iter(uploaded))
data = pd.read_excel(file_name)

# Verify all expected column names and print them
print("Columns in the dataset:", data.columns.tolist())

# Fill missing values
data.fillna(method='ffill', inplace=True, axis=0)

# Define relevant columns for processing
relevant_columns = [
    'aamc_id', 'student_id', 'student_name', 'current_status',
    'usmle1::pass_fail', 'usmle2CK::pass_fail', 'usmle2CS::pass_fail', 'usmle3::pass_fail',
    'academic_hist::extended_status', 'COEProbation::reason', 'ClassRoster::catalog_nbr',
    'ClassRoster::course_title', 'ClassRoster::official_grade', 'ClassRoster::semester_credits', 'docent',
    'OasisCourse::course', 'OasisCourse::grade'
]

# Check for missing columns
missing_columns = [col for col in relevant_columns if col not in data.columns]
if missing_columns:
    print("Missing columns:", missing_columns)
else:
    # List of courses for GPA calculation
    gpa_courses = [
        '9296', '9297', '9298', '9399', '9310', '9390', '9310', '9312', '9385', '9311',
        '9313', '9408', '9482', '9485', '9585', '9685', '9519', '9514', '9515', '9516',
        '9517', '9518', '9520', '9578', '9594'
    ]

    # Filter data to include only rows with GPA courses
    filtered_data = data[data['ClassRoster::catalog_nbr'].isin(gpa_courses)]
    filtered_data = filtered_data[relevant_columns]

    # Grade mapping for GPA calculation
    grade_mapping = {
        'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 'C+': 2.3, 'C': 2.0, 'C-': 1.7,
        'D+': 1.3, 'D': 1.0, 'D-': 0.7, 'F': 0.0, 'NR': 0.0, 'WF': 0.0, 'W': None, 'I': None,
        'AT': None, 'CR': None, 'NC': None, 'P': None, 'S': None
    }

    # GPA calculation
    filtered_data['Numeric Grade'] = filtered_data['ClassRoster::official_grade'].map(grade_mapping)
    filtered_data['Weighted Grade'] = filtered_data['Numeric Grade'] * filtered_data['ClassRoster::semester_credits']
    gpa_results = filtered_data.groupby('student_id').apply(
        lambda x: x['Weighted Grade'].sum() / x['ClassRoster::semester_credits'].sum()
    ).reset_index(name='GPA')

    # Define clinical rotations and calculate Latin Honors
    clinical_rotations = [
        'MEDIC 9571', 'MEDIC 9503', 'MEDIC 9501', 'MEDIC 9505', 'MEDIC 9506', 'MEDIC 9471',
        'MEDIC 9678', 'MEDIC 9571BR', 'MEDIC 9503BR', 'MEDIC 9501BR', 'MEDIC 9505BR',
        'MEDIC 9506BR', 'MEDIC 9678BR'
    ]

    def calculate_honors_and_latin_honors(student_id):
        # Filter honors data for the student and specific clinical rotations
        honors_data = data[(data['student_id'] == student_id) &
                           (data['OasisCourse::course'].isin(clinical_rotations)) &
                           (data['OasisCourse::grade'] == 'Honors')]

        # Remove duplicates to avoid counting the same course multiple times
        honors_data = honors_data.drop_duplicates(subset=['OasisCourse::course'])

        # Count of unique honors courses
        honors_count = len(honors_data)

        # Get GPA for the student
        gpa = gpa_results[gpa_results['student_id'] == student_id]['GPA'].values[0]

        # Assign Latin Honors based on GPA and honors count
        if gpa >= 3.750 and honors_count >= 7:
            return honors_count, 'Summa Cum Laude'
        elif gpa >= 3.600 and honors_count >= 6:
            return honors_count, 'Magna Cum Laude'
        elif gpa >= 3.500 or honors_count >= 5:
            return honors_count, 'Cum Laude'
        return honors_count, 'No Honors'

    # Calculate honors and Latin honors for each student
    honors_results = {student_id: calculate_honors_and_latin_honors(student_id)
                      for student_id in gpa_results['student_id']}

    # Create the final output DataFrame
    output_data = pd.DataFrame({
        'Student ID': gpa_results['student_id'],
        'Student Name': [data[data['student_id'] == sid]['student_name'].iloc[0] for sid in gpa_results['student_id']],
        'GPA': gpa_results['GPA'],
        'Number of Honors': [honors_results[sid][0] for sid in gpa_results['student_id']],
        'Latin Honors': [honors_results[sid][1] for sid in gpa_results['student_id']],
        'Admit Status': [data[data['student_id'] == sid]['admit_status'].iloc[0] for sid in gpa_results['student_id']]
    })

    # Round GPA to 3 decimal places
    output_data['GPA'] = output_data['GPA'].round(3)

    # Save the processed data to a new Excel file
    output_file = 'student_honors_summary.xlsx'
    output_data.to_excel(output_file, index=False)

    # Download the processed file
    files.download(output_file)

    print("Output file 'student_honors_summary.xlsx' has been created and is ready for download.")


Saving M25_student_list.xlsx to M25_student_list (2).xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Columns in the dataset: ['aamc_id', 'admit_status', 'Campus', 'citizenship', 'city', 'current_status', 'disadvantaged', 'docent', 'ethnicity', 'first_gen_student', 'gender', 'grad_indicator', 'Grad_month_tx', 'grad_year_tx', 'latin_honors', 'left_pgm_1x_level', 'left_pgm_2x_level', 'level_indicator', 'LGBT', 'mat_level', 'mat_month_tx', 'mat_year_tx', 'month_left_pgm_1x_tx', 'month_left_pgm_2x_tx', 'readmit_level', 'readmit_month_tx', 'readmit_year_tx', 'reason_left_1x', 'reason_left_2x', 'reportable_action', 'reportable_action_text', 'residency', 'ruralUrban', 'student_id', 'student_name', 'team', 'year_left_pgm_1x_tx', 'year_left_pgm_2x_tx', 'ethnicity::description', 'COEProbation::student_id', 'COEProbation::med_year', 'COEProbation::reason', 'COEProbation::prob_begin_date_mon', 'COEProbation::prob_begin_date_year', 'COEProbation::prob_end_date_mo', 'COEProbation::prob_end_date_year', 'COEProbation::Length', 'COEProbation::orig_value', 'OasisCourse::student_id', 'OasisCourse::grade'

<ipython-input-3-ef7e4edebfda>:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True, axis=0)
<ipython-input-3-ef7e4edebfda>:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gpa_results = filtered_data.groupby('student_id').apply(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Output file 'student_honors_summary.xlsx' has been created and is ready for download.
